In [1]:
%load_ext autoreload
%autoreload 2

In [127]:
# Data handling
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from pathlib import Path

import networkx as nx

import nx_utils
from nx_utils import( 
    get_weights_and_biases_from_state_dict, 
    load_state_dict_from_file, 
)
import bokeh_utils
from bokeh_utils import (
    LINE_ALPHA, 
    LINE_COLOR,
    LINE_COLOR_LIST,
    LAYER
)
from bokeh.io import output_file, output_notebook 
# output_notebook()  # Render inline in a Jupyter Notebook

In [128]:
MODELDIR = Path("models")
TIME = "2023-09-16_220748"
NAME = 'Number-1'
dir = MODELDIR/NAME/TIME

In [129]:
G = nx.DiGraph()

list_of_files = sorted(dir.glob("*.pt"))

state_dict = nx_utils.load_state_dict_from_file(list_of_files[0])
weight_shapes = nx_utils.get_shape_from_state_dict(state_dict)

nx_utils.add_neuron_nodes(G, weight_shapes)
# nx.draw(G, pos=nx.multipartite_layout(G, subset_key=LAYER))

In [132]:
weights = [
    get_weights_and_biases_from_state_dict(
        load_state_dict_from_file(f)
    )[0]
    for f in list_of_files
]
layers_of_weights = [torch.stack(x) for x in zip(*weights)]

attribute_functions = {
       LINE_COLOR_LIST : nx_utils.black_and_white(layers_of_weights)
    }

nx_utils.add_weight_edges_arrays(
    G, 
    nx_utils.get_layers_of_nodes(G), 
    attribute_functions
    )
bokeh_utils.draw_interactive_mlp_graph(G)